In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
%matplotlib inline
import os

In [2]:
## Imports the code used to calculate totals. Saved as Med_Shift_Total_release_ETL.py

import Med_Shift_Total_release_ETL as med_etl

In [3]:
med_analyze = med_etl.PainMedETL()

In [6]:
# Load in chort data to analyze - The csv file was filtered using R to show only the input data for the first 24hrs of 
# our CABG pateint cohort
# Any csv file derived from the metavision input events table in mimic can be used

med_data = pd.read_csv('MV_CABG.csv',
                       parse_dates=['STARTTIME', 'ENDTIME'])
med_data.shape

(143085, 33)

In [7]:
# Filter cohort by ID of interest, in this case we will be using the admission ID (HADM_ID)

filter_cohort = med_data['ICUSTAY_ID'].unique()
filter_cohort_list = list(filter_cohort)

In [8]:
# Create empty data frame for output

def dn_totals(med_df):
    if type(med_df.index[0]) is not pd.tslib.Timestamp:
        med_df.set_index('Shift_Start', drop=False, inplace=True)
    col_list = med_df.columns
    med_day = med_df[med_df.index.hour == 7]
    med_night = med_df[med_df.index.hour == 19]
    med_totals = {}
    for col in col_list:
        if col != 'Shift_Start':
            key1 = 'D_'+ col
            med_totals[key1] = med_day[col].sum()
            key2 = 'N_'+col
            med_totals[key2] = med_night[col].sum()
    return med_totals

In [9]:
bolus_sum_dict = {}
iv_sum_dict = {}
for i in range(10):
    enc_id = filter_cohort_list[i]
    sub_med_data = med_data.loc[med_data['ICUSTAY_ID'] == enc_id]
    med_bolus_df = sub_med_data.loc[sub_med_data['ORDERCATEGORYNAME'] == '05-Med Bolus']
    med_iv_df = sub_med_data.loc[sub_med_data['ORDERCATEGORYNAME'] == '01-Drips']
    if med_bolus_df.shape[0] > 0:
        bolus_shift_totals = med_analyze.get_bolus_shift_total(med_bolus_df)
        totals_dict = dn_totals(bolus_shift_totals)
        bolus_sum_dict[enc_id] = totals_dict
    if med_iv_df.shape[0] > 0:
        iv_shift_totals = med_analyze.get_IV_shift_total(med_iv_df)
        if iv_shift_totals.shape[0] > 0:
            total_iv_dict = dn_totals(iv_shift_totals)
            iv_sum_dict[enc_id] = total_iv_dict
    print enc_id

KeyError: 'Amount'

In [10]:
iv_sum_dict

{}

In [11]:
iv_sum_df = pd.DataFrame.from_dict(iv_sum_dict, orient='index')

In [12]:
iv_sum_df.head()

""


In [13]:
bolus_sum_dict = {}
iv_sum_dict = {}
for i in range(len(filter_cohort_list)):
    enc_id = filter_cohort_list[i]
    sub_med_data = med_data.loc[med_data['ICUSTAY_ID'] == enc_id]
    med_bolus_df = sub_med_data.loc[sub_med_data['ORDERCATEGORYNAME'] == '05-Med Bolus']
    med_iv_df = sub_med_data.loc[sub_med_data['ORDERCATEGORYNAME'] == '01-Drips']
    if med_bolus_df.shape[0] > 0:
        bolus_shift_totals = med_analyze.get_bolus_shift_total(med_bolus_df)
        totals_dict = dn_totals(bolus_shift_totals)
        bolus_sum_dict[enc_id] = totals_dict
    if med_iv_df.shape[0] > 0:
        iv_shift_totals = med_analyze.get_IV_shift_total(med_iv_df)
        if iv_shift_totals.shape[0] > 0:
            total_iv_dict = dn_totals(iv_shift_totals)
            iv_sum_dict[enc_id] = total_iv_dict
    if i%200 == 0:
        print enc_id

KeyError: 'Amount'

In [14]:
iv_sum_df = pd.DataFrame.from_dict(iv_sum_dict, orient='index')

In [15]:
bolus_sum_df = pd.DataFrame.from_dict(bolus_sum_dict, orient='index')

In [16]:
iv_sum_df.shape

(0, 0)

In [17]:
bolus_sum_df.shape

(0, 0)

In [18]:
iv_sum_df.tail()

""


In [19]:
bolus_sum_df.tail()

""
